In [1]:
import os

path = 'C:/Users/'+os.getlogin()+'/Google Drive/University/Dissertation'
#datapath = 'C:/Users/'+os.getlogin()+'/Dissertation Data'
datapath = 'E:/Dissertation Data'

os.chdir(path+'/Code')
os.getcwd()

'C:\\Users\\User\\Google Drive\\University\\Dissertation\\Code'

In [2]:
import pickle
import pandas as pd
import re
import numpy as np

import nltk
from nltk.tokenize import MWETokenizer

from nltk.corpus.reader.plaintext import PlaintextCorpusReader
    
from nltk.tokenize import WhitespaceTokenizer

from generators import sent_gen, word_gen, Sent_Seq

In [3]:
vocab = set(['April' , 'month', 'year', 'week', 'in', 'the', 'years', 'May', 'March', 'July'])

In [4]:
samp = PlaintextCorpusReader(datapath+'/Corpora/wiki/simple_20200601/','simple_sample.txt',
                            word_tokenizer = WhitespaceTokenizer()
                            )

In [5]:
for sent in sent_gen(samp):
    print(sent)

['April', 'is', 'the', 'fourth', 'month', 'of', 'the', 'year', 'and', 'comes', 'between', 'March', 'and', 'May']
['It', 'is', 'one', 'of', 'four', 'months', 'to', 'have', '30', 'days']
['April', 'always', 'begins', 'on', 'the', 'same', 'day', 'of', 'week', 'as', 'July', 'and', 'additionally', 'January', 'in', 'leap', 'years']
['April', 'always', 'ends', 'on', 'the', 'same', 'day', 'of', 'the', 'week', 'as', 'December']
["April's", 'flowers', 'are', 'the', 'Sweet', 'Pea', 'and', 'Daisy']
['Its', 'birthstone', 'is', 'the', 'diamond']
['The', 'meaning', 'of', 'the', 'diamond', 'is', 'innocence']
['The', 'Month', 'Spring', 'flowers', 'in', 'April', 'in', 'the', 'Northern', 'Hemisphere']
['April', 'comes', 'between', 'March', 'and', 'May', 'making', 'it', 'the', 'fourth', 'month', 'of', 'the', 'year']
['It', 'also', 'comes', 'first', 'in', 'the', 'year', 'out', 'of', 'the', 'four', 'months', 'that', 'have', '30', 'days', 'as', 'June', 'September', 'and', 'November', 'are', 'later', 'in', 't

In [6]:
for sent in sent_gen(samp, vocab=vocab):
    print(sent)

['April', '<unk>', 'the', '<unk>', 'month', '<unk>', 'the', 'year', '<unk>', '<unk>', '<unk>', 'March', '<unk>', 'May']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['April', '<unk>', '<unk>', '<unk>', 'the', '<unk>', '<unk>', '<unk>', 'week', '<unk>', 'July', '<unk>', '<unk>', '<unk>', 'in', '<unk>', 'years']
['April', '<unk>', '<unk>', '<unk>', 'the', '<unk>', '<unk>', '<unk>', 'the', 'week', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', 'the', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', 'the', '<unk>']
['<unk>', '<unk>', '<unk>', 'the', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', 'in', 'April', 'in', 'the', '<unk>', '<unk>']
['April', '<unk>', '<unk>', 'March', '<unk>', 'May', '<unk>', '<unk>', 'the', '<unk>', 'month', '<unk>', 'the', 'year']
['<unk>', '<unk>', '<unk>', '<unk>', 'in', 'the', 'year', '<unk>', '<unk>', 'the', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<u

['<unk>', '<unk>', '<unk>', '<unk>', 'in', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', 'in', '<unk>', '<unk>', '<unk>', 'in', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', 'in', 'July', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 'the', 'March', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', 'the', '<unk>', '<unk>', 'in', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']
['<unk>', 

In [7]:
from gensim import corpora

In [8]:
gdict = corpora.Dictionary(sent_gen(samp, vocab = vocab))

In [9]:
gdict.add_documents([['bums']])

In [10]:
print(gdict.token2id)

{'<unk>': 0, 'April': 1, 'March': 2, 'May': 3, 'month': 4, 'the': 5, 'year': 6, 'July': 7, 'in': 8, 'week': 9, 'years': 10, 'bums': 11}


In [11]:
gdict[0]

'<unk>'

In [12]:
from scipy import sparse
import itertools

In [13]:
def build_cooccur(vocab_dict, sentence, window_size=10):
    """
    Based on https://github.com/hans/glove.py/
    This version yields a (small) sparse matrix for a single sentence, in scipy's CSR format for efficient
     combination with others by summing - this will allow for distributed creation of co-occurrence matrix.
    
    vocab_dict is a gensim Dictionary mapping the vocabulary of the corpus to IDs.
    """

    vocab_size = len(vocab_dict)
    
    # Collect cooccurrences internally as a sparse matrix
    cooccurrences = sparse.lil_matrix((vocab_size, vocab_size),
                                      dtype=np.float64)

    token_ids = [vocab_dict.token2id[word] for word in sentence]

    for center_i, center_id in enumerate(token_ids):
        # Collect all word IDs in left window of center word
        context_ids = token_ids[max(0, center_i - window_size) : center_i]
        contexts_len = len(context_ids)

        for left_i, left_id in enumerate(context_ids):
            # Distance from center word
            distance = contexts_len - left_i

            # Weight by inverse of distance between words
            increment = 1.0 / float(distance)

            # Build co-occurrence matrix symmetrically (pretend we
            # are calculating right contexts as well)
            cooccurrences[center_id, left_id] += increment
            cooccurrences[left_id, center_id] += increment
            
    # Convert to CSR format for efficient combination with matrices from other sentences
    return cooccurrences.tocsr()


In [14]:
for sent in sent_gen(samp, vocab=vocab):
    print(build_cooccur(gdict, sent).toarray())
    break

[[13.51825397  1.86944444  3.225       2.00833333  3.075       5.65119048
   2.95        0.          0.          0.          0.          0.        ]
 [ 1.86944444  0.          0.          0.          0.25        0.66666667
   0.14285714  0.          0.          0.          0.          0.        ]
 [ 3.225       0.          0.          0.5         0.14285714  0.31111111
   0.25        0.          0.          0.          0.          0.        ]
 [ 2.00833333  0.          0.5         0.          0.11111111  0.14285714
   0.16666667  0.          0.          0.          0.          0.        ]
 [ 3.075       0.25        0.14285714  0.11111111  0.          1.
   0.33333333  0.          0.          0.          0.          0.        ]
 [ 5.65119048  0.66666667  0.31111111  0.14285714  1.          0.5
   1.2         0.          0.          0.          0.          0.        ]
 [ 2.95        0.14285714  0.25        0.16666667  0.33333333  1.2
   0.          0.          0.          0.          0. 

In [15]:
A = sparse.csr_matrix((len(gdict),len(gdict)), dtype=np.float64)

In [16]:
%%time

for sent in sent_gen(samp, vocab=vocab):
    A += build_cooccur(gdict, sent)
    
A.toarray()

Wall time: 601 ms


array([[1.63128135e+04, 6.41389683e+02, 6.65702381e+01, 3.87059524e+01,
        7.17051587e+01, 1.14477778e+03, 1.08444048e+02, 4.31797619e+01,
        4.93353175e+02, 9.65785714e+01, 1.37223413e+02, 0.00000000e+00],
       [6.41389683e+02, 6.44285714e+00, 5.03333333e+00, 2.63333333e+00,
        7.00000000e-01, 1.85666667e+01, 4.42857143e-01, 3.00000000e-01,
        1.93321429e+01, 1.23611111e+00, 5.98333333e+00, 0.00000000e+00],
       [6.65702381e+01, 5.03333333e+00, 0.00000000e+00, 1.61111111e+00,
        2.85714286e-01, 3.53571429e+00, 9.75000000e-01, 0.00000000e+00,
        3.95833333e+00, 1.00000000e+00, 1.40000000e+00, 0.00000000e+00],
       [3.87059524e+01, 2.63333333e+00, 1.61111111e+00, 0.00000000e+00,
        3.11111111e-01, 3.48214286e+00, 1.24166667e+00, 0.00000000e+00,
        8.33333333e-01, 1.00000000e+00, 1.28571429e+00, 0.00000000e+00],
       [7.17051587e+01, 7.00000000e-01, 2.85714286e-01, 3.11111111e-01,
        5.83333333e-01, 1.21313492e+01, 1.20000000e+00, 1.10

In [17]:
A

<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 114 stored elements in Compressed Sparse Row format>

In [18]:
sents = 0
for sent in sent_gen(samp, vocab=vocab):
    sents += 1
    
sents

459

In [19]:
from dist_cooccurrence import dist_cooccurrence, dist_cooccurrence_v2, dist_cooccurrence_v3

In [20]:
%%time

cooccurrence = dist_cooccurrence(gdict, sent_gen(samp, vocab=vocab), window_size = 10, specs=[('popen', 2)])

cooccurrence

Wall time: 3.47 s


<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 114 stored elements in Compressed Sparse Row format>

In [21]:
%%time

cooccurrence2 = dist_cooccurrence_v2(gdict, sent_gen(samp, vocab=vocab), window_size = 10, specs=[('popen', 2)])

cooccurrence2

Out of sentences - requesting returns
Out of sentences - requesting returns
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
Channel <Channel id=1 closed> terminated. 1 so far.
Channel <Channel id=1 open> terminated. 2 so far.
Everything terminated, exiting
Wall time: 2.73 s


<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 114 stored elements in Compressed Sparse Row format>

In [22]:
diffs = (cooccurrence != cooccurrence2)

In [23]:
diffs.toarray()[0]

array([ True,  True,  True,  True, False,  True,  True,  True,  True,
        True,  True, False])

In [24]:
#A.toarray()[0]

In [25]:
cooccurrence.toarray()[0]

array([16312.81349206,   641.38968254,    66.5702381 ,    38.70595238,
          71.70515873,  1144.77777778,   108.44404762,    43.1797619 ,
         493.3531746 ,    96.57857143,   137.2234127 ,     0.        ])

In [26]:
cooccurrence2.toarray()[0]

array([16312.81349206,   641.38968254,    66.5702381 ,    38.70595238,
          71.70515873,  1144.77777778,   108.44404762,    43.1797619 ,
         493.3531746 ,    96.57857143,   137.2234127 ,     0.        ])

In [27]:
%%time

cooccurrence3 = dist_cooccurrence_v3(gdict, sent_gen(samp, vocab=vocab), window_size = 10, rem_return = 1000, specs=[('popen', 4)])

cooccurrence3

Out of sentences - requesting returns
Out of sentences - requesting returns
Out of sentences - requesting returns
Obtained response from channel <Channel id=1 open>
Obtained response from channel <Channel id=1 open>
Channel <Channel id=1 closed> terminated. 1 so far.
Obtained response from channel <Channel id=1 open>
Channel <Channel id=1 closed> terminated. 2 so far.
Channel <Channel id=1 closed> terminated. 3 so far.
Out of sentences - requesting returns
Obtained response from channel <Channel id=1 open>
Channel <Channel id=1 closed> terminated. 4 so far.
Everything terminated, exiting
Wall time: 1.69 s


<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 114 stored elements in Compressed Sparse Row format>

In [28]:
cooccurrence3.toarray()[0]

array([16312.81349206,   641.38968254,    66.5702381 ,    38.70595238,
          71.70515873,  1144.77777778,   108.44404762,    43.1797619 ,
         493.3531746 ,    96.57857143,   137.2234127 ,     0.        ])

In [29]:
import glove

In [30]:
coo_dict = cooccurrence3.todok()

In [47]:
coo3 = cooccurrence3.asformat('coo')

In [48]:
coo3

<12x12 sparse matrix of type '<class 'numpy.float64'>'
	with 114 stored elements in COOrdinate format>

In [40]:
cooccurrence3.toarray()

array([[1.63128135e+04, 6.41389683e+02, 6.65702381e+01, 3.87059524e+01,
        7.17051587e+01, 1.14477778e+03, 1.08444048e+02, 4.31797619e+01,
        4.93353175e+02, 9.65785714e+01, 1.37223413e+02, 0.00000000e+00],
       [6.41389683e+02, 6.44285714e+00, 5.03333333e+00, 2.63333333e+00,
        7.00000000e-01, 1.85666667e+01, 4.42857143e-01, 3.00000000e-01,
        1.93321429e+01, 1.23611111e+00, 5.98333333e+00, 0.00000000e+00],
       [6.65702381e+01, 5.03333333e+00, 0.00000000e+00, 1.61111111e+00,
        2.85714286e-01, 3.53571429e+00, 9.75000000e-01, 0.00000000e+00,
        3.95833333e+00, 1.00000000e+00, 1.40000000e+00, 0.00000000e+00],
       [3.87059524e+01, 2.63333333e+00, 1.61111111e+00, 0.00000000e+00,
        3.11111111e-01, 3.48214286e+00, 1.24166667e+00, 0.00000000e+00,
        8.33333333e-01, 1.00000000e+00, 1.28571429e+00, 0.00000000e+00],
       [7.17051587e+01, 7.00000000e-01, 2.85714286e-01, 3.11111111e-01,
        5.83333333e-01, 1.21313492e+01, 1.20000000e+00, 1.10

In [31]:
coo_dict.items()

dict_items([((0, 0), 16312.81349206366), ((1, 0), 641.3896825396819), ((2, 0), 66.57023809523808), ((3, 0), 38.70595238095238), ((4, 0), 71.7051587301587), ((5, 0), 1144.777777777776), ((6, 0), 108.44404761904758), ((7, 0), 43.179761904761904), ((8, 0), 493.3531746031746), ((9, 0), 96.57857142857142), ((10, 0), 137.22341269841272), ((0, 1), 641.3896825396819), ((1, 1), 6.442857142857142), ((2, 1), 5.033333333333333), ((3, 1), 2.633333333333333), ((4, 1), 0.7), ((5, 1), 18.566666666666663), ((6, 1), 0.44285714285714284), ((7, 1), 0.30000000000000004), ((8, 1), 19.33214285714286), ((9, 1), 1.2361111111111112), ((10, 1), 5.983333333333334), ((0, 2), 66.57023809523808), ((1, 2), 5.033333333333333), ((3, 2), 1.6111111111111112), ((4, 2), 0.2857142857142857), ((5, 2), 3.5357142857142856), ((6, 2), 0.9749999999999999), ((8, 2), 3.958333333333333), ((9, 2), 1.0), ((10, 2), 1.4000000000000001), ((0, 3), 38.70595238095238), ((1, 3), 2.633333333333333), ((2, 3), 1.6111111111111112), ((4, 3), 0.31

In [32]:
coo_dict_final = {}

for i, v in coo_dict.items():
    if i[0] not in coo_dict_final:
        coo_dict_final[i[0]] = {}
    coo_dict_final[i[0]].update({i[1] : v})

In [33]:
coo_dict_final

{0: {0: 16312.81349206366,
  1: 641.3896825396819,
  2: 66.57023809523808,
  3: 38.70595238095238,
  4: 71.7051587301587,
  5: 1144.777777777776,
  6: 108.44404761904758,
  7: 43.179761904761904,
  8: 493.3531746031746,
  9: 96.57857142857142,
  10: 137.22341269841272},
 1: {0: 641.3896825396819,
  1: 6.442857142857142,
  2: 5.033333333333333,
  3: 2.633333333333333,
  4: 0.7,
  5: 18.566666666666663,
  6: 0.44285714285714284,
  7: 0.30000000000000004,
  8: 19.33214285714286,
  9: 1.2361111111111112,
  10: 5.983333333333334},
 2: {0: 66.57023809523808,
  1: 5.033333333333333,
  3: 1.6111111111111112,
  4: 0.2857142857142857,
  5: 3.5357142857142856,
  6: 0.9749999999999999,
  8: 3.958333333333333,
  9: 1.0,
  10: 1.4000000000000001},
 3: {0: 38.70595238095238,
  1: 2.633333333333333,
  2: 1.6111111111111112,
  4: 0.3111111111111111,
  5: 3.482142857142857,
  6: 1.2416666666666667,
  8: 0.8333333333333333,
  9: 1.0,
  10: 1.2857142857142856},
 4: {0: 71.7051587301587,
  1: 0.7,
  2: 0.2

In [34]:
model = glove.Glove(coo_dict_final, d=50, alpha=0.75, x_max=100.0)

In [35]:
for epoch in range(25):
    err = model.train(workers=9, batch_size=50)
    print("epoch %d, error %.3f" % (epoch, err), flush=True)

epoch 0, error 1.911
epoch 1, error 0.752
epoch 2, error 0.339
epoch 3, error 0.172
epoch 4, error 0.101
epoch 5, error 0.069
epoch 6, error 0.054
epoch 7, error 0.046
epoch 8, error 0.041
epoch 9, error 0.038
epoch 10, error 0.035
epoch 11, error 0.034
epoch 12, error 0.032
epoch 13, error 0.031
epoch 14, error 0.030
epoch 15, error 0.029
epoch 16, error 0.028
epoch 17, error 0.027
epoch 18, error 0.026
epoch 19, error 0.026
epoch 20, error 0.025
epoch 21, error 0.024
epoch 22, error 0.024
epoch 23, error 0.023
epoch 24, error 0.023


In [36]:
model.W

array([[-5.33982664e-02,  2.36553974e-02, -4.20766837e-02,
         1.16568438e-01,  9.48041747e-02, -4.51792760e-02,
        -1.57867023e-02,  1.13083420e-01,  1.08287644e-01,
        -2.80463873e-02, -1.52457680e-01,  8.38963841e-03,
         1.26713088e-01,  4.14446975e-03, -2.18966664e-02,
        -4.53043121e-02,  3.36241433e-02,  1.65045464e-02,
         3.08954088e-02, -1.35501196e-02,  1.21564496e-02,
         1.39305004e-01, -8.84276155e-02, -9.97151148e-02,
         2.21376387e-01,  6.19143410e-03, -4.48254986e-02,
        -2.69365208e-02, -1.90141804e-01, -7.09880622e-02,
         8.64292385e-02, -5.20101518e-02,  5.79845552e-02,
        -1.37191397e-01,  8.20267464e-02, -4.93552054e-02,
        -9.44971938e-02,  5.87625169e-02,  6.73537223e-02,
        -3.79194551e-02, -1.83134962e-01,  3.02158169e-02,
        -1.47869170e-02,  8.88531042e-02, -6.10346942e-02,
         2.07361870e-03, -1.18195554e-01,  9.67722073e-03,
        -6.85168106e-03, -1.03285357e-01],
       [-4.66

In [37]:
#  Load pickled vocab set
with open(datapath+'/Corpora/wiki/enwiki_20200520/10pc_b{}vocab.pkl'.format(1), 'rb') as pfile:
    batch_vocab = pickle.load(pfile)
    
list(batch_vocab)

['sash+windows+with',
 'Eccles+Trewavas+1989',
 'Burrus',
 'slate',
 'Weigl',
 'Doncaster+Knights',
 'Enlow',
 'and+now+has',
 "a+woman's",
 'Radio-television',
 'Grooming',
 'pre-marital',
 'grievously',
 'Jhonny',
 'cold+war',
 'Norwalk',
 'Guba',
 'GameFan',
 'kinematics',
 '53.75',
 'etiology',
 'Ishinomori',
 'incerta',
 'Bessho',
 'LGBT+community',
 'Klingenthal',
 'independently+from',
 'at+Versailles',
 'leafroller',
 'Trepča',
 'Hamann',
 'Ampon',
 'sailfish',
 'ng/mL',
 'Industrial+Research+Organisation',
 'Shaba',
 'turkey',
 'hayloft',
 'Wąbrzeźno',
 'embraced+the',
 '2007-08',
 'canisters',
 'Tuktamysheva',
 'Gradina',
 "Key's",
 'mineral+deposits',
 '1953-1957',
 'CROs',
 'Montalvo',
 'Renteria',
 'light-coloured',
 "widow's",
 'Tuckahoe',
 'commercialized',
 'cavernosa',
 'Rhadine',
 'After+his+time',
 'Filibus',
 'Rathnamala',
 'Killam',
 'Basrah',
 'is+put+on',
 'Cafeteria',
 'Rectifier',
 '19–17',
 'Specializing',
 'style+brick+dwelling',
 'Meja',
 'water+sources',
 '